# 네이버 영화평 감성분석
- Soynlp, TfidfVectorizer, Naive Bayes 

In [1]:
import numpy as np
import pandas as pd 
import warnings
warnings.filterwarnings('ignore')

In [2]:
train_df = pd.read_csv('data/naver_movie_train.tsv', sep='\t')
test_df = pd.read_csv('data/naver_movie_test.tsv', sep='\t')
train_df.head(3)

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


- Tokenizer 함수 정의

In [4]:
import joblib
scores = joblib.load('data/scores.pkl')

In [3]:
stop_text = '은 는 이 가 을 를 의 에게 에 들 좀 잘 과 도 으로 자 와 ㅋㅋ ㅠㅠ ㅎㅎ'
stopwords = stop_text.split()

In [5]:
from soynlp.tokenizer import MaxScoreTokenizer
max_tok = MaxScoreTokenizer(scores=scores)

def max_tokenizer(text):
    words = max_tok.tokenize(text)
    tokens = [word for word in words if word not in stopwords]
    return tokens

- TfidfVectorizer 로 변환

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

tvect = TfidfVectorizer(
    tokenizer=max_tokenizer, ngram_range=(1,2), max_df=0.9
)

In [7]:
%time tvect.fit(train_df.document)

Wall time: 25.9 s


TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=0.9, max_features=None,
                min_df=1, ngram_range=(1, 2), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<function max_tokenizer at 0x000001F276580B88>,
                use_idf=True, vocabulary=None)

In [8]:
%time X_train_tv = tvect.transform(train_df.document)

Wall time: 22.9 s


In [9]:
%time X_test_tv = tvect.transform(test_df.document)

Wall time: 7.54 s


In [10]:
y_train = train_df.label.values 
y_test = test_df.label.values 

- Naive Bayes 분류기

In [11]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.get_params()

{'alpha': 1.0, 'class_prior': None, 'fit_prior': True}

In [12]:
nb.fit(X_train_tv, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [13]:
nb.score(X_test_tv, y_test)

0.8554409236059937

- 실제 테스트

In [14]:
reviews = ['아름다운 음악과 아름다운 풍광 그렇지 못한 현실이 찡하네요',
           '메시지와 작위성의 불협화음']

In [15]:
reviews_tv = tvect.transform(reviews)
nb.predict(reviews_tv)

array([1, 0], dtype=int64)